In [304]:
# Import required libraries
import numpy as np
import pandas as pd 
import seaborn as sns
import sklearn.metrics as metrics
import matplotlib.pyplot as plt 
from sklearn.feature_extraction.text import CountVectorizer

In [305]:
# Read HBB and HEXA file
df5 = pd.read_excel("/home/kargas/D/dataset.xlsx")
df5 = df5.drop('Spacer sequence', axis=1)
df5.rename(columns={"3' extension":"3extension"}, inplace=True)

In [306]:
# Creating the Bag of Words model using CountVectorizer()
# This is equivalent to k-mer counting
cv = CountVectorizer(ngram_range=(3,3), analyzer='char')
X = cv.fit_transform(df5['3extension'])

In [307]:
print(cv.get_feature_names())

['aac', 'aag', 'acc', 'acg', 'act', 'aga', 'agg', 'agt', 'ata', 'atc', 'atg', 'cac', 'cag', 'ccc', 'ccg', 'cct', 'cgg', 'cgt', 'cta', 'ctc', 'ctg', 'ctt', 'gaa', 'gac', 'gag', 'gca', 'gcc', 'gga', 'ggc', 'ggt', 'gta', 'gtc', 'gtg', 'taa', 'tac', 'tat', 'tca', 'tcc', 'tct', 'tga', 'tgc', 'tgg', 'tta', 'ttc']


In [308]:
# Print the vocabulary
print('Vocabulary: ')
print(cv.vocabulary_)
print()

Vocabulary: 
{'ata': 8, 'tat': 35, 'atc': 9, 'tct': 38, 'ctt': 21, 'tta': 42, 'atg': 10, 'tgg': 41, 'ggc': 28, 'gcc': 26, 'ccc': 13, 'cct': 15, 'ctg': 20, 'tga': 39, 'gac': 23, 'act': 4, 'gga': 27, 'gaa': 22, 'gta': 30, 'acc': 2, 'ccg': 14, 'cgt': 17, 'aac': 0, 'aag': 1, 'agg': 6, 'tac': 34, 'ggt': 29, 'tcc': 37, 'cta': 18, 'ctc': 19, 'gag': 24, 'aga': 5, 'agt': 7, 'gtc': 31, 'tgc': 40, 'ttc': 43, 'tca': 36, 'cag': 12, 'gtg': 32, 'gca': 25, 'cac': 11, 'taa': 33, 'acg': 3, 'cgg': 16}



In [309]:
# Convert sparse csr matrix to dense format and allow columns to contain the array mapping from feature integer indices to feature names
count_vect_df = pd.DataFrame(X.todense(), columns=cv.get_feature_names())

In [310]:
# Concatenate the original dataframe and the count_vect_df columnwise
df5 = pd.concat([df5, count_vect_df], axis=1)

In [312]:
# Target variable
y = df5['Mean of correct edit']

In [313]:
# Drop the column that will be predicted and assign the features to variable
X = df5.drop(['pegRNA', '3extension', 'Mean of correct edit'], axis = 1)

In [314]:
# Import the Random Forest Regressor model
from sklearn.ensemble import RandomForestRegressor

In [315]:
# Splitting the dataset into the training set and test set
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.3, random_state=42)

In [316]:
print(X_train.shape)
print(X_test.shape)

(39, 46)
(18, 46)


In [317]:
print(y_train.shape)
print(y_test.shape)

(39,)
(18,)


In [227]:
# Grid search the Random Forest Regressor
from sklearn.model_selection import GridSearchCV

# Setting the static parameters
rfr = RandomForestRegressor(bootstrap=True, random_state=42, n_jobs=7)

param_grid = dict(n_estimators=[10, 25, 50, 100, 1000],
                  max_depth=[5, 10, 20, 30],
                  min_samples_leaf=[1,2,4])

grid = GridSearchCV(rfr, param_grid, cv=10,
                    scoring='neg_mean_squared_error')
grid.fit(X_train,y_train)

GridSearchCV(cv=10, estimator=RandomForestRegressor(n_jobs=7, random_state=42),
             param_grid={'max_depth': [5, 10, 20, 30],
                         'min_samples_leaf': [1, 2, 4],
                         'n_estimators': [10, 25, 50, 100, 1000]},
             scoring='neg_mean_squared_error')

In [228]:
print("grid.cv_results_ {}".format(grid.cv_results_))

grid.cv_results_ {'mean_fit_time': array([0.12463112, 0.07253141, 0.10366592, 0.14999108, 0.97920547,
       0.0346015 , 0.07394829, 0.11610303, 0.15042763, 0.98279836,
       0.04302967, 0.07481318, 0.10801034, 0.14933031, 0.96823561,
       0.04545033, 0.07898946, 0.11146545, 0.15363224, 0.9826067 ,
       0.04157143, 0.07846863, 0.11358545, 0.16034601, 0.95718281,
       0.04779823, 0.07516367, 0.11140878, 0.15930786, 0.93888125,
       0.04347451, 0.07673059, 0.11855297, 0.14848857, 0.9716327 ,
       0.04684811, 0.07258337, 0.11499126, 0.14490662, 0.9237179 ,
       0.0425796 , 0.07884119, 0.10679483, 0.15609593, 0.91978731,
       0.04268093, 0.07488394, 0.10995338, 0.14987173, 0.96735435,
       0.04265113, 0.07095132, 0.10957475, 0.15619984, 0.94978542,
       0.03629506, 0.07367566, 0.10757513, 0.14846447, 0.93551953]), 'std_fit_time': array([0.25955994, 0.012552  , 0.01170685, 0.01386467, 0.05884535,
       0.00830686, 0.01561255, 0.0110974 , 0.01187331, 0.03966606,
       0.

In [229]:
print("grid.best_params_ {}".format(grid.best_params_))
print("grid.best_estimator_ {}".format(grid.best_estimator_)) 

grid.best_params_ {'max_depth': 30, 'min_samples_leaf': 1, 'n_estimators': 1000}
grid.best_estimator_ RandomForestRegressor(max_depth=30, n_estimators=1000, n_jobs=7,
                      random_state=42)


In [230]:
# Instantiate model with grid search parameters
rf = RandomForestRegressor(bootstrap=True, ccp_alpha=0.0, criterion='mse',
                      max_depth=30, max_features='auto', max_leaf_nodes=None,
                      max_samples=None, min_impurity_decrease=0.0,
                      min_impurity_split=None, min_samples_leaf=1,
                      min_samples_split=2, min_weight_fraction_leaf=0.0,
                      n_estimators=1000, n_jobs=5, oob_score=False,
                      random_state=42, verbose=0, warm_start=False)

In [231]:
# Train the model on training data
rf.fit(X_train, y_train)

RandomForestRegressor(max_depth=30, n_estimators=1000, n_jobs=5,
                      random_state=42)

In [232]:
# Use the forest's predict method on the test data
predictions = rf.predict(X)

In [233]:
# Place predictions in the dataframe
df5['Mean of correct edit'] = df5['Mean of correct edit']
df5['Predictions'] = pd.DataFrame(data=predictions)

In [234]:
results = df5[['Mean of correct edit', 'Predictions']] 

In [235]:
# Print the actual values and the model's predictions
results

,Mean of correct edit,Predictions
0,0.244361,0.826592
1,0.200436,0.919855
2,0.974926,1.825110
3,3.501457,9.749571
4,5.617375,10.927047
5,8.081667,9.783342
6,8.507569,9.610457
7,0.295150,2.884263
8,0.217884,3.151879
9,2.557439,4.517991


In [236]:
import sklearn.metrics as metrics

# Calculate MAE, MSE, RMSE
print(metrics.mean_absolute_error(df5['Mean of correct edit'],predictions))
print(metrics.mean_squared_error(df5['Mean of correct edit'], predictions))
print(np.sqrt(metrics.mean_squared_error(df5['Mean of correct edit'], predictions)))

2.107452712413652
7.438407116580154
2.727344334069344


Gradient Boosting Regressor

In [237]:
# Import the Gradient Boosting Regressor model
from sklearn.ensemble import GradientBoostingRegressor

In [238]:
# Keep the features
X2 = X

In [239]:
# Target variable
y = df5['Mean of correct edit']

In [240]:
# Splitting the dataset into the training set and test set
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X2, y, test_size = 0.3, random_state=42)

In [241]:
#Random Search Gradient Boosting Regressor
from sklearn.model_selection import GridSearchCV

param_grid = {'learning_rate': [0.1, 0.05, 0.02, 0.01],
'max_depth': [4, 6],
'min_samples_leaf': [3, 5, 9, 17],
'max_features': [1.0, 0.3, 0.1]}

est = GradientBoostingRegressor(n_estimators=3000)
gs_cv = GridSearchCV(est, param_grid, n_jobs=4).fit(X_train, y_train)

# Best hyperparameter setting
gs_cv.best_params_

{'learning_rate': 0.05,
 'max_depth': 6,
 'max_features': 0.1,
 'min_samples_leaf': 3}

In [242]:
# Parameters from grid search
gbr = GradientBoostingRegressor(n_estimators=3000, max_depth=6, learning_rate=0.05,
                                loss='huber', max_features=0.1,
                                min_samples_leaf=3)

In [243]:
gbr.fit(X_train, y_train)

GradientBoostingRegressor(learning_rate=0.05, loss='huber', max_depth=6,
                          max_features=0.1, min_samples_leaf=3,
                          n_estimators=3000)

In [244]:
# Use the Gradient Boosting Regressor predict method on the test data
predictions2 = gbr.predict(X2)

In [245]:
# Place predictions in the dataframe
df5['Mean of correct edit'] = df5['Mean of correct edit']
df5['Predictions 2'] = pd.DataFrame(data=predictions2)

In [246]:
results2 = df5[['Mean of correct edit', 'Predictions 2']] 

In [247]:
# Print the actual values and the model's predictions
results2

,Mean of correct edit,Predictions 2
0,0.244361,-0.450940
1,0.200436,0.203824
2,0.974926,0.968245
3,3.501457,6.123273
4,5.617375,9.026403
5,8.081667,8.216805
6,8.507569,7.819625
7,0.295150,0.315396
8,0.217884,1.413076
9,2.557439,2.535665


In [248]:
import sklearn.metrics as metrics

# Calculate MAE, MSE, RMSE
print(metrics.mean_absolute_error(df5['Mean of correct edit'], predictions2))
print(metrics.mean_squared_error(df5['Mean of correct edit'], predictions2))
print(np.sqrt(metrics.mean_squared_error(df5['Mean of correct edit'], predictions2)))

0.9393936060413121
3.828396766027494
1.9566289290582142


For DMD insertion data

In [355]:
# Read DMD insertion data file
df_ins = pd.read_excel("/home/kargas/D/3_extension_DMD_exon44_insertion.xls")
extensions = df_ins["3' extension"]
df_ins = df_ins.drop('Unnamed: 0', axis=1)
df_ins = df_ins.drop('Names', axis=1)
df_ins.rename(columns={"3' extension":"3extension"}, inplace=True)

In [356]:
# Creating the Bag of Words model using CountVectorizer()
# This is equivalent to k-mer counting
X2 = cv.transform(df_ins['3extension'])

# Convert sparse csr matrix to dense format and allow columns to contain the array mapping from feature integer indices to feature names
count_vect_df2 = pd.DataFrame(X2.todense(), columns=cv.get_feature_names())

# Concatenate the original dataframe and the count_vect_df columnwise
df_ins = pd.concat([df_ins, count_vect_df2], axis=1)

# Delete string column
df_ins = df_ins.drop('3extension', axis=1)

For DMD deletion data

In [357]:
# Read DMD deletion data file
df_del = pd.read_excel("/home/kargas/D/3_extension_DMD_exon44_5-3_deletion.xls")
extensions2 = df_del["3' extension"]
df_del = df_del.drop('Unnamed: 0', axis=1)
df_del = df_del.drop('Names', axis=1)
df_del.rename(columns={"3' extension":"3extension"}, inplace=True)

In [358]:
# Creating the Bag of Words model using CountVectorizer()
# This is equivalent to k-mer counting
X3 = cv.transform(df_del['3extension'])

# Convert sparse csr matrix to dense format and allow columns to contain the array mapping from feature integer indices to feature names
count_vect_df3 = pd.DataFrame(X3.todense(), columns=cv.get_feature_names())

# Concatenate the original dataframe and the count_vect_df columnwise
df_del = pd.concat([df_del, count_vect_df3], axis=1)

# Delete string column
df_del = df_del.drop('3extension', axis=1)

Test algorithms on the DMD data

In [359]:
# Use the forest's predict method on the test data
predictions_ins = rf.predict(df_ins)

# Use the Gradient Boosting Regressor predict method on the test data
predictions_ins2 = gbr.predict(df_ins)

# Place predictions in new columns
df_ins['rfr predictions'] = predictions_ins
df_ins['gbr predictions'] = predictions_ins2
df_ins["3' extension"] = extensions

In [360]:
# Use the forest's predict method on the test data
predictions_del = rf.predict(df_del)

# Use the Gradient Boosting Regressor predict method on the test data
predictions_del2 = gbr.predict(df_del)

# Place predictions in new columns
df_del['rfr predictions'] = predictions_del
df_del['gbr predictions'] = predictions_del2
df_del["3' extension"] = extensions2

In [364]:
# Write predictions to excel
df_ins.to_excel('df_ins_predictions.xlsx')
df_del.to_excel('df_del_predictions.xlsx')